In [1]:
import tensorflow as tf

C:\Users\krishna\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("/MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
n_classes = 10

batch_size = 128

In [5]:
x = tf.placeholder('float', [None, 784])


In [6]:

#Creating weights and biases

w = tf.Variable(tf.zeros([784,10]))

b = tf.Variable(tf.zeros([10]))

In [7]:
# our model

y = tf.nn.softmax(tf.matmul(x,w) + b)

In [8]:
#training our model

y_ = tf.placeholder(tf.float32, [None,10])

#cross entropy function

#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))

In [9]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [10]:

init = tf.global_variables_initializer()

In [11]:
sess = tf.Session()

sess.run(init) 

for i in range(10000):

    batch_xs, batch_ys = mnist.train.next_batch(100)

    sess.run(train_step, feed_dict={x : batch_xs, y_: batch_ys})

#Evaluating the model:

prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.925


In [12]:
#  Multilayer Convolutional Layer:

#functions to define weights and biases to be used in the computation ahead.

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
    
  return tf.Variable(initial)


In [13]:
def bias_variable(shape):

  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [14]:
def conv2d(x, W):

  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [15]:

def max_pool_2x2(x):

  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                    
    strides=[1, 2, 2, 1], padding='SAME')

In [16]:

get_ipython().run_line_magic('pinfo', 'tf.nn.conv2d')

In [17]:
#First Convolutional Layer

W_conv1 = weight_variable([5, 5, 1, 32])

b_conv1 = bias_variable([32])

In [18]:
x_image = tf.reshape(x, [-1,28,28,1])

In [19]:
x_image.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(1)])

In [20]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

h_pool1 = max_pool_2x2(h_conv1)
b_conv2 = bias_variable([64])




In [21]:
#Second Convolutional Layer

W_conv2 = weight_variable([5, 5, 32, 64])

b_conv2 = bias_variable([64])



h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

h_pool2 = max_pool_2x2(h_conv2)

In [22]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])

b_fc1 = bias_variable([1024])

In [23]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
print(h_pool2_flat.shape)

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

(?, 3136)


In [24]:
#Dropout Layer

keep_prob = tf.placeholder(tf.float32)
print(keep_prob)

h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Tensor("Placeholder_2:0", dtype=float32)


In [25]:
#Readout Layer

W_fc2 = weight_variable([1024, 10])

b_fc2 = bias_variable([10])

In [ ]:
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
with tf.Session() as sess:





    cross_entropy = tf.reduce_mean(

        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))

    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    sess.run(tf.global_variables_initializer())



    for i in range(20000):

      batch = mnist.train.next_batch(50)

      if i%100 == 0:

        train_accuracy = accuracy.eval(feed_dict={

            x:batch[0], y_: batch[1], keep_prob: 1.0})

        print("step %d, training accuracy %g"%(i, train_accuracy))

      train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})



    print("test accuracy %g"%accuracy.eval(feed_dict={

        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

step 0, training accuracy 0.12
step 100, training accuracy 0.84
step 200, training accuracy 0.88
step 300, training accuracy 0.92
step 400, training accuracy 0.92
step 500, training accuracy 0.96
step 600, training accuracy 0.98
step 700, training accuracy 1
step 800, training accuracy 1
step 900, training accuracy 0.92
step 1000, training accuracy 0.96
step 1100, training accuracy 0.98
step 1200, training accuracy 0.96
step 1300, training accuracy 0.98
step 1400, training accuracy 0.96
step 1500, training accuracy 0.96
step 1600, training accuracy 0.98
step 1700, training accuracy 0.98
step 1800, training accuracy 1
step 1900, training accuracy 0.92
step 2000, training accuracy 0.98
step 2100, training accuracy 0.94
step 2200, training accuracy 0.98
step 2300, training accuracy 0.98
step 2400, tra